# Call WorkGraph inside WorkChain

## Introduction
In this tutorial, we will show how to call a WorkGraph inside a WorkChain.




In [2]:
from aiida.engine import WorkChain, calcfunction, run
from aiida import orm, load_profile

load_profile()


@calcfunction
def add(x, y):
    return x + y


@calcfunction
def multiply(x, y):
    return x * y

from aiida_workgraph import WorkGraph

wg = WorkGraph("add_multiply_workflow")
add_task = wg.tasks.new(add, name="add")
# link the output of the `add` task to one of the `x` input of the `multiply` task.
wg.tasks.new(multiply, name="multiply", x = add_task.outputs["result"])
wg.group_outputs = [{"name": "multiply", "from": "multiply"}]
wg.tasks["add"].set({'x': 1, 'y': 2})
wg.tasks["multiply"].set({'y': 2})

class AddAndMultiplyWorkChain(WorkChain):
    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input_namespace('wgdata', dynamic=True)
        spec.outline(
            cls.run_workgraph,
            cls.results,
        )
        spec.output('result')

    def run_workgraph(self):
        from aiida_workgraph.engine.workgraph import WorkGraphEngine
        inputs = {
            'wg': self.inputs.wgdata,
        }
        print("submit workgraph: ")
        process = self.submit(WorkGraphEngine, inputs=inputs)
        self.to_context(workgraph=process)

    def results(self):
        self.out('result', self.ctx.workgraph.outputs.multiply.result)
    

run(AddAndMultiplyWorkChain, inputs={"wgdata": wg.to_dict()})

submit workgraph: 


07/14/2024 03:25:41 AM <2859676> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [89920|WorkGraphEngine|continue_workgraph]: Continue workgraph.
07/14/2024 03:25:41 AM <2859676> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [89920|WorkGraphEngine|continue_workgraph]: tasks ready to run: add
07/14/2024 03:25:41 AM <2859676> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [89920|WorkGraphEngine|run_tasks]: Run task: add, type: CALCFUNCTION


Continue workgraph.
------------------------------------------------------------
executor:  <function add at 0x7288f50994e0>
task type: calcfunction/workfunction.


07/14/2024 03:25:42 AM <2859676> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [89920|WorkGraphEngine|run_tasks]: Task: add finished.
07/14/2024 03:25:42 AM <2859676> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [89920|WorkGraphEngine|continue_workgraph]: Continue workgraph.
07/14/2024 03:25:42 AM <2859676> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [89920|WorkGraphEngine|continue_workgraph]: tasks ready to run: multiply
07/14/2024 03:25:42 AM <2859676> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [89920|WorkGraphEngine|run_tasks]: Run task: multiply, type: CALCFUNCTION


Continue workgraph.
------------------------------------------------------------
executor:  <function multiply at 0x7288f509b880>
task type: calcfunction/workfunction.


07/14/2024 03:25:42 AM <2859676> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [89920|WorkGraphEngine|run_tasks]: Task: multiply finished.
07/14/2024 03:25:42 AM <2859676> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [89920|WorkGraphEngine|continue_workgraph]: Continue workgraph.
07/14/2024 03:25:42 AM <2859676> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [89920|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
07/14/2024 03:25:42 AM <2859676> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [89920|WorkGraphEngine|finalize]: Finalize


Continue workgraph.
task:  add FINISHED
task:  multiply FINISHED
is workgraph finished:  True
workgraph outputs:  [{'from': 'multiply', 'name': 'multiply'}]
output:  {'from': 'multiply', 'name': 'multiply'}
Finalize workgraph add_multiply_workflow



{'result': <Int: uuid: f60795e5-22b2-4ac1-bc2c-4ec3bcdff96f (pk: 89927) value: 6>}